In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
def one_hot_encode(data):

  # 分類數量
  num_classes = 10
  # min_birth_year = min(data['birth_year'])
  # max_birth_year = max(data['birth_year'])
  # min_birth_year = data['birth_year'].astype(int).min()
  # max_birth_year = data['birth_year'].astype(int).max()
  # 取得所有非空白年份的最小值
  min_birth_year = data.loc[data['birth_year']!=' ', 'birth_year'].min()

  # 取得所有非空白年份的最大值
  max_birth_year = data.loc[data['birth_year']!=' ', 'birth_year'].max()

  interval = (max_birth_year - min_birth_year) // num_classes
  birth_year_categories = []
  for year in data['birth_year']:
    if type(year)==str:
      birth_year_categories.append(" ")
    else:
      category =  int((year - min_birth_year) // interval)
      birth_year_categories.append(category)

  gender_categories = ['Male', 'Female']
  zodiac_categories = ['Aquarius', 'Pisces', 'Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn']
  state_categories = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
  education_categories = ['Elementary school', 'Middle school', 'High School', 'University', 'Graduate School','Master']

  # 將各特徵進行 One-Hot Encoding
  #gender_one_hot = tf.one_hot(data['gender'].apply(lambda x: tf.zeros((len(gender_categories),)) if x == ' ' else gender_categories.index(x)), depth=len(gender_categories))
  gender_one_hot = tf.one_hot(
    indices=[gender_categories.index(x) if x != " " else -1 for x in data['gender']],
    depth=len(gender_categories),
    on_value=1,
    off_value=0,
  )
  #print(gender_one_hot)
  zodiac_one_hot = tf.one_hot(
    indices=[zodiac_categories.index(x) if x != " " else -1 for x in data['zodiac']],
    depth=len(zodiac_categories),
    on_value=1,
    off_value=0,
  )

  state_one_hot = tf.one_hot(
    indices=[state_categories.index(x) if x != " " else -1 for x in data['state']],
    depth=len(state_categories),
    on_value=1,
    off_value=0,
  )

  education_one_hot = tf.one_hot(
    indices=[education_categories.index(x) if x != " " else -1 for x in data['education']],
    depth=len(education_categories),
    on_value=1,
    off_value=0,
  )

  birth_year_one_hot = tf.one_hot(
    indices=[x-1 if x != " " else -1 for x in birth_year_categories],
    depth=num_classes,
    on_value=1,
    off_value=0, 
  )
 
  # 將 Tensor 轉換為 Numpy Array
  gender_one_hot_np = np.array(gender_one_hot)
  birth_year_one_hot_np = np.array(birth_year_one_hot)
  zodiac_one_hot_np = np.array(zodiac_one_hot)
  state_one_hot_np = np.array(state_one_hot)
  education_one_hot_np = np.array(education_one_hot)

  # 將所有特徵合併
  all_features = np.concatenate([gender_one_hot_np, birth_year_one_hot_np, zodiac_one_hot_np, state_one_hot_np, education_one_hot_np], axis=1)

  print(all_features)
  print(all_features.shape)

  return all_features

In [50]:

# 資料
data = pd.read_csv('output.csv')

# data = pd.DataFrame({
#     'gender': ['male', 'female', 'male', 'female', 'male'],
#     'birth_year': [1990, 1985, 1995, 1980, 2000],
#     'zodiac': ['Aquarius', 'Taurus', 'Leo', 'Gemini', 'Virgo'],
#     'state': ['California', 'New York', 'Texas', 'Florida', 'Illinois'],
#     'education': ['University', 'Graduate School', 'High school', 'Middle school', 'Elementary school']
# })
features = ['gender', 'birth_year', 'zodiac', 'state', 'education']
x=data[features]
x = x.fillna(' ')

features = ['gender', 'birth_year', 'zodiac', 'state', 'education']
x = data[features]
x = x.fillna(' ')

data['birth_year'] = pd.to_numeric(data['birth_year'], errors='coerce')

print(x)
data_one_hot=one_hot_encode(x)

print(data_one_hot)




     gender birth_year  zodiac           state    education
0      Male                           Illinois   University
1      Male                       Pennsylvania  High School
2                          Leo           Texas       Master
3      Male                           Arkansas   University
4      Male     1990.0  Pisces    Pennsylvania             
..      ...        ...     ...             ...          ...
297    Male              Virgo      New Mexico             
298  Female                     North Carolina   University
299    Male                            Florida   University
300    Male                           Missouri   University
301    Male                           Missouri   University

[302 rows x 5 columns]
[[1 0 0 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 1 0 0]
 [1 0 0 ... 1 0 0]
 [1 0 0 ... 1 0 0]]
(302, 80)
[[1 0 0 ... 1 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 1 0 0]
 [1 0 0 ... 1 0 0]
 [1 0 0 ... 1 0 0]]


In [51]:
import csv
#紀錄one-hot檔案
with open('one_hot.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in data_one_hot:
        writer.writerow(row)

161

161

161

161

161

...

161

161

161

161

161

In [53]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

class PersonalityPredictModel:

  def __init__(self):
    # 設置輸入和目標變量
    self.features = ['gender', 'birth_year', 'zodiac', 'state', 'education']
    self.bigfive = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']
    self.model=None

  def build_model(self, X):
    # 建立模型
    self.model = Sequential()
    self.model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
    self.model.add(Dense(64, activation='relu'))
    self.model.add(Dense(5, activation='softmax'))

    # 編譯模型
    self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  def train_model(self, df, batch_size, epochs):
    # 將輸入資料和標籤分開
    X = df[self.features]
    y = df[self.bigfive]

    # 切割資料集為訓練集和驗證集
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # 訓練模型
    self.model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))

    # 評估模型準確度
    test_loss, test_acc = self.model.evaluate(X_val, y_val)
    print('Test accuracy:', test_acc)

    return test_loss, test_acc


In [ ]:
# 讀取 CSV 檔案
df = pd.read_csv('output.csv')

# 將類別型資料轉換成 one-hot encoding
df = one_hot_encode(df)

In [ ]:
#找最佳
with open('results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['batch_size', 'epochs', 'test_loss', 'test_accuracy'])
    
    for bs in batch_sizes:
        for e in epochs:
            model = PersonalityPredictModel()
            model.build_model(df)
            test_loss, test_acc = model.train_model(df, batch_size=bs, epochs=e)
            writer.writerow([bs, e, test_loss, test_acc])
